In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image

# Load the dataset directly from GitHub
democracy_data_url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2024/2024-11-05/democracy_data.csv'
democracy_data = pd.read_csv(democracy_data_url)

In [ ]:
df = democracy_data
df.loc[df['colony_of'].notna(), 'regime_category'] = 'Colony'

# Lista de regimes válidos
regimes = [
    "Civilian dictatorship",
    "Colony",
    "Military dictatorship",
    "Mixed democratic",
    "Parliamentary democracy",
    "Presidential democracy",
    "Royal dictatorship"
]

# Atualiza regime_category para "Other" onde o valor não está na lista regimes
df.loc[~df['regime_category'].isin(regimes), 'regime_category'] = 'Other'

df = df[['country_code','year','regime_category']]

# Filtrando os dados de 1950 a 2020
years = range(1950, 2021)
yearly_dataframes = {}

for year in years:
    # Filtra o DataFrame para o ano específico
    yearly_dataframes[year] = df[df['year'] == year]

In [ ]:
font = gpd.read_file("data/ne_10m_admin_0_countries.geojson")

In [ ]:
background_color = '#D9D9D9'

def create_graph_year(font,yearly_dataframes,year):
    # Mantém apenas as colunas relevantes e renomeia para combinar com o DataFrame 'df'
    world = font[['ADM0_A3', 'geometry']].rename(columns={'ADM0_A3': 'country_code'})

    # Realiza a junção com base no código do país (country_code)
    world = world.merge(yearly_dataframes[year][['country_code', 'year', 'regime_category']], on='country_code', how='left')

    # Dicionário de cores para cada tipo de regime com tons escuros
    regime_group_colors = {
        "Ditadura": "#B27A6A",       
        "Democracia": "#54879A",     
        "Colônia": "#B2946F",        
        "Outro": "#5C917B"           
    }

    # Função para categorizar regimes
    def categorize_regime(regime):
        if regime in ["Civilian dictatorship", "Military dictatorship", "Royal dictatorship"]:
            return "Ditadura"
        elif regime in ["Mixed democratic", "Parliamentary democracy", "Presidential democracy"]:
            return "Democracia"
        elif regime in ['Colony']:
            return "Colônia"
        else:
            return "Outro"

    # Aplica a função ao DataFrame
    world['regime_group'] = world['regime_category'].apply(categorize_regime)

    # Contar os regimes agrupados
    regime_group_counts = world['regime_group'].value_counts()

    total_countries = regime_group_counts.sum()

    # Cria uma nova coluna com as cores baseadas no regime
    world['color'] = world['regime_group'].map(regime_group_colors)

    # Cria subplots: um para o mapa e outro para o gráfico de barras
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 15),facecolor=background_color, gridspec_kw={'height_ratios': [4, 1]})

    # Plota o mapa colorido
    world.plot(ax=ax1, color=world['color'], edgecolor='black', linewidth=0.1)  

    ax1.set_aspect('equal')

    # Set the background color for each axes
    ax1.set_facecolor(background_color)  # Background color for the map
    ax2.set_facecolor(background_color)  # Background color for the bar chart

    # Modifica as bordas do gráfico (spines)
    for spine in ax1.spines.values():
        spine.set_color(background_color)  # Cor da borda do mapa
        spine.set_linewidth(1)  # Largura da borda do mapa

    # Modifica as bordas do gráfico (spines)
    for spine in ax2.spines.values():
        spine.set_color(background_color)  # Cor da borda do mapa
        spine.set_linewidth(1)  # Largura da borda do mapa

    ax1.set_xticks([])  # Remove ticks do eixo X
    ax1.set_yticks([])  # Remove ticks do eixo Y

    ax1.set_xlim(world.total_bounds[[0, 2]])  # Limites do eixo x do mapa
    ax1.set_ylim(world.total_bounds[[1, 3]])  # Limites do eixo y do mapa

    # Plota o gráfico de barras horizontal
    colors = regime_group_counts.index.map(regime_group_colors)  # Mapeia as cores correspondentes
    bars = regime_group_counts.plot(kind='barh', ax=ax2, color=colors, edgecolor='black', linewidth=0.4)  # Alterar a cor do contorno do gráfico
    ax2.set_xlabel('Número de Países', fontsize=16,color='black', fontdict={'fontname': 'Segoe UI', 'weight': 'light'})
    ax2.set_ylabel('Categoria de Regime', fontsize=16,color='black', fontdict={'fontname': 'Segoe UI', 'weight': 'light'})
    ax2.set_xlim(0, max(regime_group_counts) + 20)

    # Remove os rótulos do eixo y
    ax2.set_yticklabels([])

    # Adiciona porcentagens no gráfico de barras
    for bar in bars.patches:
        percentage = (bar.get_width() / total_countries) * 100
        regime_name = bar.get_y() + bar.get_height() / 2  # Obtém o índice do regime
        regime_label = regime_group_counts.index[int(regime_name)]  # Obtém o nome do regime correspondente
        ax2.text(
            bar.get_width() + 0.5,  # Largura da barra + um pequeno espaçamento
            bar.get_y() + bar.get_height() / 2,  # Centraliza verticalmente
            f'{regime_label}: {percentage:.1f}%',  # Adiciona o nome do regime e a porcentagem
            va='center',  # Alinhamento vertical
            fontsize=20,
            color='black',
            fontdict={'fontname': 'Segoe UI', 'weight': 'light'}  # Define a fonte para as anotações
        )

    ax1.set_title(f'Mapa Mundial por Tipo de Regime - {year}', fontsize=24, color='black', fontdict={'fontname': 'Segoe UI', 'weight': 'light'}, pad=15)
    plt.subplots_adjust(hspace=-0.5)
    plt.subplots_adjust(top=1)
    plt.tight_layout()

    plt.savefig(f'output/{year}.png')
    plt.close(fig)  # Close the figure to avoid display issues later

In [ ]:
# Create frames for GIF
years = range(1950, 2021)
image_files = []

for year in years:
    create_graph_year(font, yearly_dataframes, year)
    image_files.append(f'output/{year}.png')  # Store the filenames for GIF creation

# Carregar as imagens
frames = [Image.open(image) for image in image_files]

# Salvar como GIF
gif_filename = 'output/regime_maps.gif'
frames[0].save(
    gif_filename,
    save_all=True,
    append_images=frames[1:],  # Adiciona os quadros subsequentes
    duration=500,             # Duração em milissegundos (1000ms = 1 segundo)
    loop=0                     # 0 para loop infinito
)

print(f'GIF salvo como {gif_filename}')

GIF salvo como output/regime_maps.gif
